In [1]:
import os
from flask import Flask, render_template, request, redirect, url_for
from langchain.agents import initialize_agent, Tool
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from werkzeug.utils import secure_filename
import pandas as pd
from openpyxl import load_workbook

e:\TechM\.venv\Lib\site-packages\langchain\chains\api\base.py:56: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_community.utilities.requests import TextRequestsWrapper
e:\TechM\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
app = Flask(__name__)

In [3]:
@app.route('/')
def home():
    return render_template('index.html')

In [4]:
os.environ["GOOGLE_API_KEY"] = "Your gemini key here"

In [5]:
llm_flash = ChatGoogleGenerativeAI(model='gemini-1.5-flash')
llm_pro=ChatGoogleGenerativeAI(model='gemini-1.5-flash-8b')

In [6]:


def field_extraction(prompt):
    query = (
        f"Your task is to extract the following values from the given text: 1. Budget, 2. Duration, 3. Season, 4. Travel style (like adventure, family, cultural), 5. Food preference, 6. International destination or not, 7. Departure location. "
        "Extract only the values mentioned for these fields. Do not include any additional text or information. "
        "Very important: You should only extract the required fields and nothing else."
    )
    result = llm_flash.invoke(f"{query} Input: {prompt}")
    extracted_info=result.content
    info_list = extracted_info.strip().split('\n')
    extracted_data = {
        "Budget": info_list[0],
        "Duration": info_list[1],
        "Season": info_list[2],
        "Travel Style": info_list[3],
        "Food Preference": info_list[4],  
        "International": info_list[5],
        "Departure Location": info_list[6]
    }
    df = pd.DataFrame([extracted_data])  
    filename = "extracted_fields.xlsx"
    try:
        book = load_workbook(filename)
        existing_df = pd.read_excel(filename)
        updated_df = pd.concat([existing_df, df], ignore_index=True)
        updated_df.to_excel(filename, index=False)
        print("Data updated in 'extracted_fields.xlsx'")
    except FileNotFoundError:
        df.to_excel(filename, index=False)
        print(f"Data saved to a new file: '{filename}'")



In [7]:
def validate_prompt_recommendation(prompt):
    query=(
        f"Your task is to validate the {prompt} if user has provided these info or not",
        "Information to validate: Budget, Duration, season, travel style (like adventure, family, cultural), food prefrence, international destination or not, departure location",
        "Just validate it on these info ask nothing else"
    )
    validated=llm_flash.invoke(query)
    return validated

def recommendation_prompt(prompt):
    query = (
        f"Your task is to provide top 5 location to visit based on {prompt}, nothing else."
    )
    response = llm_flash.invoke(query)  
    return response

In [8]:
prompt_validation_recommendation_tool=Tool(
    name='Prompt Validation Tool',
    func=validate_prompt_recommendation,
    description='The task it to validate if the prompt contains all the necessary information to predict travel cost or not'
)
recommendation_tool=Tool(
    name='Travel Cost Prediction Tool',
    func=recommendation_prompt,
    description='The task is to recommend places based on the prompt provided by the user'
)

In [9]:
def validate_prompt_prediction(prompt):
    query=(
        f"Your task is to validate the {prompt} if user has provided these info or not",
        "Information to validate: Departure and Arrival country Mode of transport, Number of Days, Number of family members, Type of Hotel, Time of the year travelling if even one of these information is missing just return not validated",
        "Just provide output as validated or not validated nothing else",
    )
    validated=llm_flash.invoke(query)
    return validated

def predict_travel_cost(prompt):
    query = (
        f"Your task is to give an estimated cost based on {prompt}, nothing else.",
        "Very Important (Just predict a estimated cost. Predict cost in every case once the prompt is validated nothing else just estimated price)"
    )
    response = llm_flash.invoke(query)  
    return response

In [10]:
prompt_validation_prediction_tool = Tool(
    name='Prompt Validation Tool',
    func=validate_prompt_prediction,
    description='The task is to validate if the prompt contains all the necessary information to predict travel cost or not'
)
cost_prediction_tool = Tool(
    name='Travel Cost Prediction Tool',
    func=predict_travel_cost,
    description='The task is to predict the total cost of travelling from one place to another based on the prompt provided by the user'
)

In [11]:
def validate_image(image_data):
    message = HumanMessage(
    content=[
        {"type": "text", "text": "Your task is to validate the image provided to you if the image if there are existing tourist destination on earth similar to image. VERY IMPORTATNT (make sure to answer only in yes or no)"},
        {
            "type": "image_url",
            "image_url": {"url": f"data:image/jpeg;base64,{image_data}"},
        },
    ],
    )
    response = llm_pro.invoke([message])
    return response.content
def recommend_places_image(image_data):
    message = HumanMessage(
    content=[
        {"type": "text", "text": "Your task is to recommend similar places to travel according to the image provided below. Give top 5 similar places matching to this just give these places as output nothing else."},
        {
            "type": "image_url",
            "image_url": {"url": f"data:image/jpeg;base64,{image_data}"},
        },
    ],
    )
    response = llm_pro.invoke([message])
    return response.content


In [12]:
import base64
def read_image(path):
    image_path = path
    with open(image_path, 'rb') as image_file:
        image_data = base64.b64encode(image_file.read()).decode("utf-8")
    return image_data

In [13]:
memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=3,
    return_messages=True
)
conversational_agent_recommendation = initialize_agent(
    agent='chat-conversational-react-description',
    tools=[prompt_validation_recommendation_tool, recommendation_tool],
    llm=llm_flash,
    verbose=True,
    max_iterations=3,
    memory=memory
)
conversational_agent_prediction = initialize_agent(
    agent='chat-conversational-react-description',
    tools=[prompt_validation_prediction_tool, cost_prediction_tool],
    llm=llm_flash,
    verbose=True,
    max_iterations=3,
    memory=memory
)


C:\Users\ishaa\AppData\Local\Temp\ipykernel_52796\1141510932.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(
C:\Users\ishaa\AppData\Local\Temp\ipykernel_52796\1141510932.py:6: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use :meth:`~Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc.` instead.
  conversational_agent_recommendation = initialize_agent(


In [14]:
@app.route('/', methods=['GET', 'POST'])
def index():
    result = None
    if request.method == 'POST':
        prompt = request.form['prompt']
        validation_result = conversational_agent_recommendation.run(f"Validate the prompt if this contains all the info as provided to the tool: {prompt}")
        if "not" in validation_result:
            result = f"Prompt validation failed. Please provide more details. {validation_result}"
        else:
            field_extraction(prompt)
            result = conversational_agent_recommendation.run(f"Now recommend places according to tool: {prompt} ")
    return render_template('index.html', result=result)

@app.route('/costprediction',methods=['GET','POST'])
def costprediction():
    result = None
    if request.method == 'POST':
        prompt = request.form['prompt']
        validation_result = conversational_agent_prediction.run(f"Validate the prompt: {prompt}")
        if "not" in validation_result:
            result= f"Prompt validation failed. Please provide more details. {validation_result}"
        else:
            result = conversational_agent_prediction.run(f"Predict the cost for: {prompt} as the validation tool already validated it")
    return render_template('costprediction.html', result=result)

@app.route('/recommendationusingimage', methods=['GET', 'POST'])
def recommendationusingimage():
    result = None
    if request.method == 'POST':
        prompt = request.form.get('prompt', '')  
        if 'image' in request.files:
            image_file = request.files['image']
            if image_file and image_file.filename:
                filename = secure_filename(image_file.filename)
                image_data = base64.b64encode(image_file.read()).decode("utf-8")
                validation_result = validate_image(image_data)
                if "yes" in validation_result.lower() or "validated" in validation_result.lower():
                    prediction_result = recommend_places_image(image_data)
                    result = prediction_result
                else:
                    result = f"Image validation failed: {validation_result}"
            else:
                result = "No image uploaded."
    
    return render_template('uploadimage.html', result=result)


def ask_gemini(question):
    answer = llm_flash.invoke(question)
    return answer  
@app.route('/chatbot', methods=['GET', 'POST'])
def chatbot():
    result = None
    if request.method == 'POST':
        prompt = request.form['prompt']  # Get the user input from the form
        if prompt.strip().lower() == "end":
            result = "Ending the conversation. Goodbye!"
            return render_template('chatbot.html', result=result)
        else:
            response = ask_gemini(prompt)  # Call the Gemini API with the prompt
            result = response.content
            return render_template('chatbot.html', result=result)
    
    return render_template('chatbot.html')

In [ ]:
if __name__ == '__main__':
    app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [19/Oct/2024 22:06:46] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2024 22:06:47] "GET /favicon.ico HTTP/1.1" 404 -
C:\Users\ishaa\AppData\Local\Temp\ipykernel_52796\3077077752.py:6: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  validation_result = conversational_agent_recommendation.run(f"Validate the prompt if this contains all the info as provided to the tool: {prompt}")




> Entering new AgentExecutor chain...
```json
{
 "action": "Prompt Validation Tool",
 "action_input": "recommend me places to visit for adventure during summer from rishikesh no international travel with a budget of 50k for a week and the food preference is veg only"
}
```
Observation: content='The information provided is sufficient for validation. It includes:\n\n* **Budget:** 50k\n* **Duration:** 1 week\n* **Season:** Summer\n* **Travel Style:** Adventure\n* **Food Preference:** Veg\n* **International Travel:** No\n* **Departure Location:** Rishikesh \n' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': '

127.0.0.1 - - [19/Oct/2024 22:07:14] "POST / HTTP/1.1" 200 -


```json
{
  "action": "Final Answer",
  "action_input": "Based on your budget, time frame, and preference for adventure travel in the summer, here are some places you can visit from Rishikesh, focusing on vegetarian options: \n\n* **Valley of Flowers National Park:** A stunning trek with vibrant wildflowers, located in Uttarakhand, reachable from Rishikesh. You can find vegetarian food at guesthouses and local restaurants along the trek.\n* **Jim Corbett National Park:**  Explore the wildlife in this national park, famous for its tigers, elephants, and other wildlife. Numerous vegetarian options are available at resorts and hotels within the park.\n* **Auli:**  Known for its scenic beauty and skiing opportunities during winters, Auli offers stunning mountain views and adventure activities in the summer. You can find vegetarian food at restaurants and hotels in Auli.\n* **Chopta:**  A scenic village in Uttarakhand, known for its breathtaking views of the Himalayas and trekking trails. Y

...

127.0.0.1 - - [19/Oct/2024 22:09:22] "POST /costprediction HTTP/1.1" 200 -


```json
{
    "action": "Final Answer",
    "action_input": "The prompt 'I want to travel from paris to rome' is not validated, meaning it doesn't contain enough information to predict the cost of travel.  To get a cost prediction, please provide more details about your trip, such as travel dates, mode of transport, and accommodation preferences."
}
```

> Finished chain.


> Entering new AgentExecutor chain...
```json
{
    "action": "Prompt Validation Tool",
    "action_input": "I want to travel from new york to delhi for 4 days with a family of 4 by flight next summer and want to  rent ut airbnb"
}
```

Observation: content='validated \n' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM

127.0.0.1 - - [19/Oct/2024 22:09:37] "POST /costprediction HTTP/1.1" 200 -


```json
{
    "action": "Final Answer",
    "action_input": "Based on the information provided, the estimated cost of traveling from New York to Delhi for 4 days with a family of 4 by flight next summer and renting an Airbnb would be between $6,000 and $8,000."
}
```

> Finished chain.


127.0.0.1 - - [19/Oct/2024 22:10:00] "GET /chatbot HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2024 22:10:21] "POST /chatbot HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2024 22:10:22] "POST /chatbot HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2024 22:10:32] "POST /chatbot HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2024 22:10:47] "GET / HTTP/1.1" 200 -
